# Keras ile Duygu Analizi

## Verisetini aşağıdaki adresden indirin

[Twitter Sentiment Analysis Training Corpus](http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/)

[Twitter Sentiment Analysis Training Corpus Dataset](http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip)   
    

In [5]:
import pickle
import json
from keras.models import model_from_json
import pandas as pd
import numpy as np
import random

In [6]:
# Metni sayıya dönüştürmek için harf bazlı bir method kullandık
# chars.pkl harflerin değerleri ve ençok kaç harfden oluşacağı tutuluyor.
fp = open('chars.pkl','rb')
data = pickle.load(fp)
char_indices = data[0]
maxlen = data[1]
charlen = len(char_indices)

In [27]:
print 'En çok harf sayısı : ',maxlen
print 'Harfler : ',char_indices.keys()[-10:],' ...'

En çok harf sayısı :  150
 Harfler :  [']', 'a', '\xe2', 'e', 'i', 'm', 'q', 'u', 'y', '}']  ...


In [7]:
# Model ağ yapısı ve ağırlıklarını okuyoruz
model_name = 'sentiment_bilstm'
jstr = json.loads(open(model_name+'.json').read())
model = model_from_json(jstr)
model.load_weights(model_name+'.h5')

In [8]:
# Modelimizin özetine bakalım
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 150, 278)          310248    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 278)               464816    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 558       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 775,622
Trainable params: 775,622
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Veri setimizi test amaçlı okuyalım
df = pd.read_csv('Sentiment Analysis Dataset.csv')

In [10]:
# Veri setindeki olumu (1) ve olumsuz (0) veri sayısı
df.Sentiment.value_counts()

1    790184
0    788440
Name: Sentiment, dtype: int64

In [13]:
# Veri setimizden bir örnek görelim
df.sample(20)

,ItemID,Sentiment,SentimentSource,SentimentText
272679,272682,1,Sentiment140,@jggube And PayPal as a way to pay for a subsc...
315039,315042,1,Sentiment140,@kinagrannis haha thank you kina! &lt;3 made m...
107887,107890,1,Sentiment140,@darklingwater We don't know if @tomfelton The...
220299,220302,0,Sentiment140,@jennyjonasxx nooo just last day of exams
222049,222052,0,Sentiment140,@jessie_ho i havent got any rudd dollars eith...
465515,465518,0,Sentiment140,@travisking I do....but I can't sleep tonight
1487457,1487461,1,Sentiment140,Leaving Magnetic Island after a great day. Man...
1106246,1106250,1,Sentiment140,Playing POker On FACEBOOK..
564594,564598,0,Sentiment140,@xxmiana I miss you!!
130685,130688,1,Sentiment140,"@chrwel02 The recipe looks awesome, I'm gonna ..."


In [14]:
# Verilen bir metinden maxlen x charlen  ebatında bir matris üretiyor
def get_matrix(text):
    text = text.lower()
    X = np.zeros((maxlen,charlen ),float)
    for i, char in enumerate(text):
        if i >= maxlen :
            break
        if char in char_indices:
            X[i,char_indices[char]] = 1
    return X

In [29]:
ntrue = 0
kitem = 300
# Veri setinin %80 eğitim için kullanıldı
split_point = int(0.8*len(df))
for j in range(kitem):
    # Test için Eğitimde kullanılmayan verilerden rastgele seçiyoruz
    ix = random.randint(split_point,len(df)-1)
    text = df.SentimentText[ix]
    label = df.Sentiment[ix]
    print '\n',text,' -> ',label 
    dx = get_matrix(text).reshape((1,maxlen,charlen))
    dy = model.predict(dx)
    out = np.argsort(-dy)
    dy = dy/dy.sum()
    top1 = (label == out[0][0])
    if top1 :
        ntrue +=1
    print 'Predictions : ',top1
    for i in range(2):
        ix = out[0][i]
        print '   ',ix,' : ',dy[0][ix]   
        
print 'Accuracy : %.2d'%(100.0*ntrue/kitem)


Gaaa interview over. Now to wait.   ->  0
Predictions :  True
    0  :  0.706302
    1  :  0.293698

I wish Nick and Miley were niley again   ->  1
Predictions :  True
    1  :  0.532728
    0  :  0.467272

With my baby she's flipping me off   ->  0
Predictions :  True
    0  :  0.845678
    1  :  0.154322

waiting for someone who want to be my best friends   ->  0
Predictions :  False
    1  :  0.659039
    0  :  0.340961

what a fabulous day   ->  1
Predictions :  True
    1  :  0.998317
    0  :  0.00168307

Was hoping you'd prove everyone  wrong. Instead I was the one proven wrong. I won't lie - it makes me sadder than you will ever know.   ->  0
Predictions :  True
    0  :  0.742273
    1  :  0.257727

missing issybee   ->  0
Predictions :  True
    0  :  0.937942
    1  :  0.0620577

http://twitpic.com/6bt12 - Preletly nam nad domem balony   ->  1
Predictions :  True
    1  :  0.642945
    0  :  0.357055

Need to summon the enthusiasm to clean the cupboard and put it all back t

Climbing, mountain biking, good people, good food. Hoooooray for summer!!   ->  1
Predictions :  True
    1  :  0.638121
    0  :  0.361879

Today is totally not doing my father justice. I think driving past a HAPPY FATHER'S DAY sign would have been a highlight for him   ->  0
Predictions :  True
    0  :  0.563898
    1  :  0.436102

@Twist1986 I get cred!! I was so excited when I saw it, and had no one to squee with   ->  0
Predictions :  True
    0  :  0.978667
    1  :  0.0213329

wants to be back in greece soooo bad!   ->  0
Predictions :  True
    0  :  0.944344
    1  :  0.0556561

Good morning  it's a B-E-A-UTIFUL day! Enjoy the sunshine everyone! Good luck in your exams guys xoxo  ->  1
Predictions :  True
    1  :  0.994061
    0  :  0.00593861

im a love man,call me a love man....otis reading   ->  1
Predictions :  True
    1  :  0.936893
    0  :  0.063107

is tired from a long great day at Branson's!!!     ->  1
Predictions :  True
    1  :  0.514517
    0  :  0.485483

Si

With strange thoughts and strange feelings...   ->  0
Predictions :  True
    0  :  0.881896
    1  :  0.118103

getting the kids ready to go to grans for a sleepover...wine ahoy this evening.!!! wey hey..!!! but not looking forward to working 2moro   ->  0
Predictions :  True
    0  :  0.992029
    1  :  0.00797129

Sick day  watching stuff on TVshack and reading Batman comics  ->  0
Predictions :  True
    0  :  0.971039
    1  :  0.0289605

Yup thats the 1   ->  1
Predictions :  True
    1  :  0.863123
    0  :  0.136877

Goin bbqing today at Princess Place preserve.   ->  1
Predictions :  True
    1  :  0.957963
    0  :  0.0420372

The Conduits release is now July 10th? That sucks and blows   ->  0
Predictions :  True
    0  :  0.988427
    1  :  0.0115725

watching a cheerleading competition on ESPN2. happy days   ->  1
Predictions :  True
    1  :  0.998464
    0  :  0.00153548

does anyone have experience in leveling a hillside into a yard?  the house we're interested in has no

    0  :  0.594891
    1  :  0.405109

WoW is broken, and thus so is my heart.    ->  0
Predictions :  True
    0  :  0.946824
    1  :  0.0531758

is not a twitter whore..   ->  1
Predictions :  False
    0  :  0.809249
    1  :  0.190751

Wishing I was going to Lebanon with Nivine   ->  0
Predictions :  True
    0  :  0.983801
    1  :  0.0161992

what is up with my cat lately and biting me?? Really worried about him b/c I haven't been home much lately.   ->  0
Predictions :  True
    0  :  0.999037
    1  :  0.000962874

wishing my days off were the same days that people wanted to party.   ->  0
Predictions :  True
    0  :  0.84388
    1  :  0.15612

http://bit.ly/sHpet   Cherry-Cherry Boom-Boom   ->  1
Predictions :  True
    1  :  0.946935
    0  :  0.0530655

i miss my friends  i wanna go to school!!  ->  0
Predictions :  True
    0  :  0.997035
    1  :  0.00296529

Busy weekend. Barely tweeted at all.   I wish I could afford a Blackberry, LOL!  ->  0
Predictions :  True
    0 

In [23]:
"""
Got the Macbook Pro seller to send me the serial number of the machine before I paid. 
Turns out its just a macbook so not buying it now
"""

text = u"Got the Macbook Pro seller to send me the serial number of the machine"
dx = get_matrix(text).reshape((1,maxlen,charlen))
dy = model.predict(dx)
out = np.argsort(-dy)
dy = dy/dy.sum()
print 'Twit : ',text
print 'Predictions : ',out[0][0]
for i in range(2):
    ix = out[0][i]
    print '   ',ix,' : ',dy[0][ix] 
    
text_full = "Got the Macbook Pro seller to send me the serial number of the machine before I paid. Turns out its just a macbook so not buying it now"
dx = get_matrix(text_full).reshape((1,maxlen,charlen))
dy = model.predict(dx)
out = np.argsort(-dy)
dy = dy/dy.sum()
print 'Twit : ',text_full
print 'Predictions : ',out[0][0]
for i in range(2):
    ix = out[0][i]
    print '   ',ix,' : ',dy[0][ix]     

Twit :  Got the Macbook Pro seller to send me the serial number of the machine
Predictions :  1
    1  :  0.62573
    0  :  0.37427
Twit :  Got the Macbook Pro seller to send me the serial number of the machine before I paid. Turns out its just a macbook so not buying it now
Predictions :  0
    0  :  0.623167
    1  :  0.376833
